In [7]:
import numpy as np
import pandas as pd
from numba import jit

from sklearn.metrics import roc_auc_score

In [8]:
@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

In [9]:
y_true = np.random.randint(0,2,1000000)
y_pred = np.random.rand(1000000)

auc_1 = fast_auc(y_true, y_pred)
auc_2 = roc_auc_score(y_true, y_pred)

print('auc_1 :', auc_1)
print('auc_2:', auc_2)

auc_1 : 0.49970595719957556
auc_2: 0.4997059571995756


In [10]:
%timeit fast_auc(y_true, y_pred)

461 ms ± 28.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit roc_auc_score(y_true, y_pred)

1.28 s ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


fast_auc比roc_auc_score速度快一倍